In [59]:
import numpy as np
import wfdb
import glob
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pywt
import pywt.data
import os
from pathlib import Path
from PIL import Image,ImageEnhance
from scipy import fftpack
import shutil
import pandas as pd
import random
import scipy.signal as signal
from wfdb import processing
from itertools import combinations
import ast
import wfdb
#from scipy.signal.filter_design import zpk2sos

Extract Lead v1 from the 12-Leads ECG Dataset

### 먼저 모든 데이터 한 파일에 몰아넣음

In [ ]:
def Get_LeadV1_Numpy():

    for myloop in range(1, 21838, 1):
        five_digit_value = '%05d' % myloop
        signals, fields = wfdb.rdsamp("/home/ines/code/ptb-xl/data/" + five_digit_value + "_lr", channels=[6],sampto=1000)

        signals = signals.tolist()
        # print(signals)

        newest = [i[0] for i in signals]
        X = [i for i in newest if i != 0.0]
        X = np.array(X)
        np.save("/home/ines/code/ptb-xl/data_LeadV1/" + five_digit_value + '.npy', X)  # save

Get_LeadV1_Numpy()

In [59]:
def Numpy_Denoise():

    # 신호의 평균 절대 편차를 계산
    def madev(d, axis=None):
        """ Mean absolute deviation of a signal """
        return np.mean(np.absolute(d - np.mean(d, axis)), axis)

    # 웨이블릿 변환을 사용하여 신호에서 노이즈를 제거하는 함수
    # 쌍직교 함수
    # 레벨1로 웨이블릿 변환
    def wavelet_denoising(x, wavelet='bior3.1', level=1):
        # coeff: 웨이블릿 변환을 통해 계산된 근사 계수와 세부 계수를 저장하는 변수
        coeff = pywt.wavedec(x, wavelet, mode="per")
        # 세부 계수의 평균 절대 편차를 계산하여 노이즈의 크기를 추정하는 데 사용
        sigma = (1 / 0.6745) * madev(coeff[-level])
        # uthresh: 임계값으로, 노이즈의 크기를 기반으로 계산
        uthresh = sigma * np.sqrt(2 * np.log(len(x)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
        # Multilevel reconstruction using waverec
        # wavelet: Wavelet object or name string
        return pywt.waverec(coeff, wavelet, mode='per')


    src_dir = "/home/ines/code/ptb-xl/data_LeadV1/"  # 21837
    dest_dir = "/home/ines/code/ptb-xl/data__Denoised/"

    # 반복문을 통해 모든 .npy 파일에 대해 노이즈 제거를 수행합니다. 각 파일을 웨이블릿 변환을 통해 필터링하고 새로운 파일로 저장
    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]

        signal = np.load(file)

        # wavelist=['bior1.3','bior2.8','bior3.1','bior3.9','db2','db8','rbio1.5','sym6','sym8']
        wavelist = ['bior3.1']


        for wav in wavelist:
            filtered = wavelet_denoising(signal, wavelet=wav, level=1)
            np.save(dest_dir+new_file_name+'.npy', filtered) # save

Numpy_Denoise()

Frequency Filtration to reduce data size

### 한 번에 돌리기엔 너무 많아서 따로 파일 두고 4000개 씩 돌림

- 기존 설정으로 train 결과 확인 후 figsize=(64,64) 해보기

In [ ]:
def Draw_Spectrogram():
    
    src_dir = "/home/ines/code/ptb-xl/data__filtered/21837/"
    dest_dir = "/home/ines/code/ptb-xl/data__spec/imshitest/"


    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:7]   

        #Loading file
        x = np.load(file)   

        #applying STFT transformation
        f, t, Zxx = signal.stft(x, nfft=9, nperseg=9, noverlap=5, window='hann') #NFFT calculate by N base Log 2 with x=len(x) 
        plt.figure(figsize=(1.15, 1.19))

        #plotting spectrogram
        plt.pcolormesh(t, f, np.abs(Zxx), cmap="gray_r")
        plt.axis("off")
        plt.savefig(dest_dir+ new_file_name + '.png',bbox_inches='tight',pad_inches=0)
        plt.close('all')

Draw_Spectrogram()

Raw Signal Graph


**Raw signal일단 스킵**

In [ ]:
# fig size 수정 필요

def DrawGraphs():

    src_dir = "/home/ines/code/ptb-xl/data__filtered/16000/12000/"
    dest_dir = "/home/ines/code/ptb-xl/data__gr/"


    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:7]  

        # Loading file
        x = np.load(file)  

        plt.figure(figsize=(5, 5))   
        plt.plot(x, 'gray')
        plt.axis("off")

        # saving plots
        plt.savefig(dest_dir + new_file_name + '.jpeg')
        plt.close('all')

        
DrawGraphs()

Two Data Augmentation Approaches on Spectrogram / Raw Signal Images

In [ ]:
import os
import shutil
import pandas as pd
import ast  

def distribute_images():
    test_fold = 10
    
    # 데이터베이스(csv 파일) 로드
    df_db = pd.read_csv('/home/ines/code/ptb-xl/ptbxl_database.csv', index_col='ecg_id')
    # ASMI를 포함하는 인덱스
    asmi_indices = df_db[df_db['scp_codes'].str.contains('ASMI', na=False)].index
    # ASMI가 아닌 인덱스
    normal_indices = df_db[~df_db.index.isin(asmi_indices)].index

    df_st = pd.read_csv('/home/ines/code/ptb-xl/scp_statements.csv', index_col=0)
    df_st = df_st[df_st.diagnostic == 1]


    source_dir = f"/home/ines/code/ptb-xl/data__spec/"
    train_dir_asmi = f'/home/ines/code/ptb-xl/X_train/asmi/'
    train_dir_normal = f'/home/ines/code/ptb-xl/X_train/normal/'
    test_dir_asmi = f'/home/ines/code/ptb-xl/X_test/asmi/'
    test_dir_normal = f'/home/ines/code/ptb-xl/X_test/normal/'
    
    # ASMI 이미지 이동
    for i in asmi_indices:
        five_digit_value = '%05d' % i
        file_name = five_digit_value + ".png"
        if df_db.loc[i, 'strat_fold'] != test_fold:
            destination_path = os.path.join(train_dir_asmi, file_name)
        else:
            destination_path = os.path.join(test_dir_asmi, file_name)
        shutil.move(os.path.join(source_dir, file_name), destination_path)

    # ASMI가 아닌 이미지 이동
    for n in normal_indices:
        five_digit_value = '%05d' % n
        file_name = five_digit_value + ".png"
        if df_db.loc[n, 'strat_fold'] != test_fold:
            destination_path = os.path.join(train_dir_normal, file_name)
        else:
            destination_path = os.path.join(test_dir_normal, file_name)
        shutil.move(os.path.join(source_dir, file_name), destination_path)


# 이미지 분배 함수 호출
distribute_images()

**Augmentation**

In [ ]:
def HorizontalFlip():
    src_dir = "/home/ines/code/ptb-xl/X_train/asmi/"
    dest_dir = "/home/ines/code/ptb-xl/X_train/asmi_aug/"

    for file in glob.iglob(src_dir + '**/*.png', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]
        print(file)
        #opening Image and convert into 2-D
        img = Image.open(file)
        img = np.array(img)

        # Flipping images with Numpy
        flipped_img = np.fliplr(img)

        img=Image.fromarray(flipped_img)
        img.save(dest_dir+new_file_name+"_hFlip"+".png")
        plt.close('all')

def SetContrast():
    src_dir = "/home/ines/code/ptb-xl/X_train/asmi/"
    dest_dir = "/home/ines/code/ptb-xl/X_train/asmi_aug/"

    for file in glob.iglob(src_dir + '**/*.png', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]

        # read the image
        im = Image.open(file)

        # image brightness enhancer
        enhancer = ImageEnhance.Contrast(im)

        factor = 1.7  # increase contrast
        im_output = enhancer.enhance(factor)
        im_output.save(dest_dir + new_file_name + "_Contrast" + ".png")


HorizontalFlip()
SetContrast()

In [ ]:
def HorizontalFlip():
    src_dir = "/home/ines/code/ptb-xl/X_train/asmi/"
    dest_dir = "/home/ines/code/ptb-xl/X_train/asmi_aug/"

    for file in glob.iglob(src_dir + '**/*.png', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]
        print(file)
        #opening Image and convert into 2-D
        img = Image.open(file)
        img = np.array(img)

        # Flipping images with Numpy
        flipped_img = np.fliplr(img)

        img=Image.fromarray(flipped_img)
        img.save(dest_dir+new_file_name+"_hFlip"+".png")
        plt.close('all')

def SetContrast():
    src_dir = "/home/ines/code/ptb-xl/X_train/asmi/"
    dest_dir = "/home/ines/code/ptb-xl/X_train/asmi_aug/"

    for file in glob.iglob(src_dir + '**/*.png', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]

        # read the image
        im = Image.open(file)

        # image brightness enhancer
        enhancer = ImageEnhance.Contrast(im)

        factor = 1.7  # increase contrast
        im_output = enhancer.enhance(factor)
        im_output.save(dest_dir + new_file_name + "_Contrast" + ".png")


HorizontalFlip()
SetContrast()

### 전처리 끝

In [49]:
'''

import os
import shutil
import pandas as pd
import ast  

def distribute_images():
    test_fold = 10
    
    # 데이터베이스(csv 파일) 로드
    df_db = pd.read_csv('/home/ines/code/ptb-xl/ptbxl_database.csv', index_col='ecg_id')
    # ASMI를 포함하는 인덱스
    asmi_indices = df_db[df_db['scp_codes'].str.contains('ASMI', na=False)].index
    # ASMI가 아닌 인덱스
    normal_indices = df_db[~df_db.index.isin(asmi_indices)].index

    
    df_st = pd.read_csv('/home/ines/code/ptb-xl/scp_statements.csv', index_col=0)
    df_st = df_st[df_st.diagnostic == 1]



    # diagnostic superclass 추가 

    source_dir = f"/home/ines/code/ptb-xl/data__spec/"
    asmi_dir = f"/home/ines/code/ptb-xl/d/asmi/"
    normal_dir = f"/home/ines/code/ptb-xl/d/normal/"

    for i in asmi_indices:
        five_digit_value = '%05d' % i
        file_name = five_digit_value + ".png"   
        destination_path = os.path.join(asmi_dir, file_name)
        shutil.move(os.path.join(source_dir, file_name), destination_path)

    for i in normal_indices:
        five_digit_value = '%05d' % i
        file_name = five_digit_value + ".png"        
        destination_path = os.path.join(normal_dir, file_name)
        shutil.move(os.path.join(source_dir, file_name), destination_path)
    

# 이미지 분배 함수 호출
distribute_images()
'''

Index([   63,   106,   131,   162,   177,   181,   182,   184,   189,   223,
       ...
       21700, 21736, 21742, 21762, 21764, 21765, 21778, 21780, 21796, 21805],
      dtype='int64', name='ecg_id', length=2357)


In [25]:
import pandas as pd

# 데이터베이스(csv 파일) 로드
df_db = pd.read_csv('/home/ines/code/ptb-xl/ptbxl_database.csv', index_col='ecg_id')

# ASMI를 포함하는 인덱스
asmi_indices = df_db[df_db['scp_codes'].str.contains('ASMI', na=False)].index
# ASMI가 아닌 인덱스
normal_indices = df_db[~df_db.index.isin(asmi_indices)].index

print(asmi_indices)


Index([   63,   106,   131,   162,   177,   181,   182,   184,   189,   223,
       ...
       21700, 21736, 21742, 21762, 21764, 21765, 21778, 21780, 21796, 21805],
      dtype='int64', name='ecg_id', length=2357)


In [26]:
print(normal_indices)

Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,    10,
       ...
       21828, 21829, 21830, 21831, 21832, 21833, 21834, 21835, 21836, 21837],
      dtype='int64', name='ecg_id', length=19442)

In [27]:
intersection = set(asmi_indices) & set(normal_indices)

if intersection:
    print("중복되는 인덱스가 있습니다.")
    print("중복되는 인덱스:", intersection)
else:
    print("중복되는 인덱스가 없습니다.")


중복되는 인덱스가 없습니다.


In [62]:
def Freq_Filteration():
    #### https://scipy-lectures.org/intro/scipy/auto_examples/plot_fftpack.html

    src_dir = "/home/ines/code/ptb-xl/data__Denoised/"
    dest_dir = "/home/ines/code/ptb-xl/data__filtered/"

    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):

        #Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:7]

        #Load the file
        x = np.load(file)  # 17876_lr

        # The FFT of the signal
        sig_fft = fftpack.fft(x)

        # The corresponding frequencies
        sample_freq = fftpack.fftfreq(x.size, d=0.02)

        #The original signal copy
        high_freq_fft = sig_fft.copy()
        high_freq_fft = np.delete(high_freq_fft, np.where(np.abs(sample_freq > 2)))

        #Taking Inverse of FFT
        filtered_sig = fftpack.ifft(high_freq_fft)

        #Saving resultant filtered signal into npy
        np.save(dest_dir+new_file_name+".npy", filtered_sig)
    
Freq_Filteration()

541
537
528
536
538
541
539
533
541
535
535
541
538
535
541
541
525
538
532
534
537
532
530
539
531
538
541
537
532
537
536
537
537
535
539
519
531
538
541
537
536
528
541
533
537
530
538
539
539
541
538
536
530
537
535
538
537
537
538
539
538
539
541
522
537
533
539
539
537
538
538
533
539
541
539
541
539
534
536
541
537
532
534
529
538
539
541
534
539
539
537
539
541
541
536
537
538
538
535
536
539
541
537
531
535
530
537
532
538
539
530
541
533
538
530
533
539
538
526
535
533
516
538
541
539
537
533
533
539
538
541
537
538
535
528
531
541
536
535
536
534
537
539
535
537
539
541
535
541
533
541
539
538
541
537
529
538
538
535
541
533
529
534
535
539
536
539
528
538
536
537
539
539
538
536
533
539
537
535
523
536
532
537
537
537
533
538
535
538
541
537
528
538
538
541
536
538
538
538
531
537
538
532
541
537
537
538
538
539
537
538
541
538
536
539
541
537
506
539
531
535
529
535
541
536
539
538
541
534
530
526
534
538
539
518
541
538
537
539
539
535
537
535
538
541
539
529
528
538
532


Spectrograms Using STFT (Scipy Library)